In [1]:
## Generate a graph
using StatsBase
using Combinatorics

using Pkg; Pkg.activate(".")
using HypergraphModularity

 Activating environment at `~/hypergraph_modularities_code/Project.toml`


In [2]:
n = 200
Z = rand(1:2, n)
ϑ = dropdims(ones(1,n) + rand(1,n), dims = 1)

# defining group intensity function Ω
μ = mean(ϑ)

ω(p,α) = (10 .*μ*sum(p))^(-sum(p))*prod(p.^α)^(1/(sum(p)*α))
α0 = 1

kmax = 3

Ω = partitionIntensityFunction(ω, kmax)
## Sample
H = sampleSBM(Z, ϑ, Ω; α=α0, kmax=kmax, kmin = 1)

hypergraph
  N: Array{Int64}((200,)) [1, 2, 3, 4, 5, 6, 7, 8, 9, 10  …  191, 192, 193, 194, 195, 196, 197, 198, 199, 200]
  E: Dict{Int64,Dict}
  D: Array{Int64}((200,)) [4, 5, 14, 4, 7, 9, 0, 3, 10, 8  …  8, 5, 4, 3, 5, 5, 12, 4, 11, 6]


In [4]:
## Run Louvains, Naive, and Faster version

# @time Zlou = Naive_HyperLouvain(H,Ω;α=α0)
@time Zlou2 = HyperLouvain(H,kmax,Ω;α=α0)

# mlou = modularity(H,Zlou,Ω,α=α0)
mlou2 = modularity(H,Zlou2,Ω,α=α0)

# This may not be exactly the same, but that's okay.
# Even a difference of 1e-14 in modularity change may lead you to make a 
# different greedy decision, but it's still a valid move.
# @show mlou, mlou2
# @show abs(mlou- mlou2)

Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
  0.396903 seconds (3.61 M allocations: 169.197 MiB, 15.29% gc time)


-45609.9503941100948587115859376127369184672488700016401708126068115234375

In [6]:
## Try supernode Louvain
# Sanity check: having a singletons initialization is equivalent to naive louvain
Z = collect(1:n)
@time Znew, changed = SuperNodeStep(H,Z,kmax,Ω;α=α0)
mlou3 =  modularity(H,Znew,Ω,α=α0)

toc = 0.003702878952026367
Louvain Iteration 1
Louvain Iteration 2
t1 = 0.34953784942626953
  0.360575 seconds (2.26 M allocations: 127.317 MiB, 12.17% gc time)


-45609.5042985878798322500529411997749473783869689214043319225311279296875

In [7]:
## Now repeatedly call the SuperNodeStep until no more improvement is possible
@time Z = SuperNodeLouvain(H,kmax,Ω;α=α0)
msuper = modularity(H,Z,Ω,α=α0)

Faster SuperNode Louvain: Phase 1
Louvain Iteration 1
Louvain Iteration 2
Louvain Iteration 3
SuperNode Louvain: Phase 2
toc = 0.002927064895629883
Louvain Iteration 1
t1 = 0.14384770393371582
  0.637081 seconds (4.86 M allocations: 239.968 MiB, 14.13% gc time)


-45609.9503941100948587115859376127369184672488700016401708126068115234375